In [1]:
!pip install tokenizer
!pip install transformers
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# https://scottmduda.medium.com/generating-an-edgar-allen-poe-styled-poem-using-gpt-2-289801ded82c


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

import os

C:\Users\lubna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {
    'bos_token': '<BOS>', 
    'eos_token': '<EOS>', 
    'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [4]:
MAX_LEN = 1024

In [12]:
!dir

 Volume in drive D is New Volume
 Volume Serial Number is 5A59-A9C0

 Directory of d:\python project\poetry

21-12-2023  22:23    <DIR>          .
18-12-2023  15:29    <DIR>          ..
18-12-2023  20:56    <DIR>          archive
18-12-2023  20:56    <DIR>          archive (2)
18-12-2023  21:55            16,719 gpt_2_fine_tuning_w_hugging_face_&_pytorch.py
22-12-2023  20:15           103,675 modelLoading.ipynb
21-12-2023  22:14       510,415,797 poem_stanza_model.pth
21-12-2023  21:38            13,289 poetry.ipynb
19-12-2023  23:00            14,964 PoetryGPT.ipynb
               5 File(s)    510,564,444 bytes
               4 Dir(s)   3,497,156,608 bytes free


In [13]:
!pip install -U huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_LEN).from_pretrained('gpt2', output_hidden_states=True)

poem_stanza_model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)

poem_stanza_model.resize_token_embeddings(len(tokenizer))

# poem_stanza_model.load_state_dict(torch.load("poem_stanza_model.pth", map_location=torch.device('cpu')))
# poem_stanza_model.load_state_dict(torch.load("poem_stanza_model.pth"))
# poem_stanza_model.load_state_dict(torch.load("poem_stanza_model.pth", map_location=torch.device('cpu')))


Embedding(50260, 768)

In [20]:
model_path = "model/poem_stanza_model.pth"

In [21]:
import os
os.path.exists(model_path)

True

In [24]:
from transformers import AutoTokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

OSError: model/poem_stanza_model.pth is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [22]:
poem_stanza_model.from_pretrained(model_path)

OSError: model/poem_stanza_model.pth is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [17]:
# text = "I love my dog"
# input_ids = tokenizer.encode(text, return_tensors='tf')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

poem_stanza_model = poem_stanza_model.to(device)

# create text generation seed promp
prompt = "<BOS> Please god"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

sample_outputs = poem_stanza_model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length=600,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

# print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Please god be my father, God be my father in this city. Go up. I can see you kneeling there, my son. Go up. I can see you kneeling in his pants. Then turn around now, father. Don't kneel on me. Go up. I can tell your son what I feel. Let me know what he feels.You have no father, no mother, no daughter, no daddy. Nothing.


1: Please god let us make our own. Donations to some of us can be made in whole cloth, in a wooden box and in paper, with no care that you may find something in it; no more than you might bear, in your own hand, the mark of a woman or a stranger in the midst of a storm."


2: Please god keep her from murdering us by telling her what I am going to do, so much better to be killed by watching her take my pictures so they get to watch her kill the people who know what she knows, and think she wants to make me kill her.She will do what she feels like if I


